<a href="https://colab.research.google.com/github/mlaricobar/datathon-interbank-2019-solution/blob/master/%5B03%5D%20Feat%20Engineering%20-%202020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering

In [1]:
from google.colab import drive, files

import pandas as pd

from datetime import datetime
from dateutil import relativedelta

In [2]:
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
%cd /gdrive/'My Drive'/Competencias/'[02] Interbank-2020'
!ls -lh '[00] Data'

/gdrive/My Drive/Competencias/[02] Interbank-2020
total 5.1G
-rw------- 1 root root 979M Dec 12 01:33 censo_test.csv
-rw------- 1 root root 748M Dec 12 01:34 censo_train.csv
-rw------- 1 root root  998 Dec 12 01:34 productos.csv
-rw------- 1 root root 1.7G Dec 12 01:34 rcc_test.csv
-rw------- 1 root root 1.4G Dec 12 01:35 rcc_train.csv
-rw------- 1 root root 3.3M Dec 12 01:35 sample_submission.csv
-rw------- 1 root root  30M Dec 12 01:35 se_test.csv
-rw------- 1 root root  28M Dec 12 01:35 se_train.csv
-rw------- 1 root root 122M Dec 12 01:35 sunat_test.csv
-rw------- 1 root root 106M Dec 12 01:35 sunat_train.csv
-rw------- 1 root root 3.0M Dec 12 01:35 y_train.csv


## Dataset RCC

In [4]:
def optimizar_df(df):
  
  for col_ in df.select_dtypes(include=[int]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="unsigned")

  for col_ in df.select_dtypes(include=[float]).columns:
    df[col_] = pd.to_numeric(df[col_], downcast="float")
  
  return df

In [5]:
df_rcc_train = optimizar_df(pd.read_csv("[00] Data/rcc_train.csv"))
df_rcc_test = optimizar_df(pd.read_csv("[00] Data/rcc_test.csv"))

In [6]:
df_rcc_train.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30391626 entries, 0 to 30391625
Data columns (total 9 columns):
 #   Column                    Non-Null Count     Dtype  
---  ------                    --------------     -----  
 0   codmes                    30391626 non-null  uint32 
 1   key_value                 30391626 non-null  uint32 
 2   condicion                 30391626 non-null  uint16 
 3   tipo_credito              30391626 non-null  uint8  
 4   cod_instit_financiera     30391626 non-null  uint8  
 5   saldo                     30391626 non-null  float32
 6   PRODUCTO                  30391626 non-null  uint8  
 7   RIESGO_DIRECTO            30391626 non-null  int64  
 8   COD_CLASIFICACION_DEUDOR  30391626 non-null  uint8  
dtypes: float32(1), int64(1), uint16(1), uint32(2), uint8(4)
memory usage: 753.6 MB


In [7]:
df_rcc_train.head()

,codmes,key_value,condicion,tipo_credito,cod_instit_financiera,saldo,PRODUCTO,RIESGO_DIRECTO,COD_CLASIFICACION_DEUDOR
0,201711,4,0,12,33,-0.072601,6,-1,0
1,201711,4,0,12,33,-0.070608,7,-1,0
2,201711,4,0,12,61,-0.052054,5,-1,0
3,201711,4,0,11,32,-0.072456,8,1,0
4,201711,4,0,12,61,-0.057877,0,1,0


In [8]:
month_list = sorted(df_rcc_train["codmes"].unique().tolist(), reverse=True)

In [9]:
df_rcc_train_agg = (df_rcc_train.groupby(["codmes", "key_value"], as_index=False)
                                .agg({
                                        "condicion": ["mean", "count"],
                                        "saldo": ["mean"]
                                      })
                    )

df_rcc_train_agg.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_rcc_train_agg.columns]

In [10]:
df_rcc_train_agg

,codmes,key_value,condicion_mean,condicion_count,saldo_mean
0,201703,0,0.0,4,-0.055520
1,201703,2,0.0,3,-0.070903
2,201703,5,0.0,6,-0.029645
3,201703,6,0.0,3,-0.065808
4,201703,7,0.0,14,-0.058202
...,...,...,...,...,...
3501370,201802,358482,0.0,3,-0.065319
3501371,201802,358483,0.0,3,-0.069119
3501372,201802,358484,0.0,4,-0.064931
3501373,201802,358485,0.0,1,-0.058336


In [11]:
top_freq_products = [1, 2, 3, 8, 4, 0, 6]
top_propensity_products = [12, 17, 18, 25, 20]
low_propensity_products = [32, 33, 35, 36, 37, 38, 40, 41]
list_product = top_freq_products + top_propensity_products + low_propensity_products
print(list_product)

[1, 2, 3, 8, 4, 0, 6, 12, 17, 18, 25, 20, 32, 33, 35, 36, 37, 38, 40, 41]


In [27]:
cat_col = "PRODUCTO"
df_rcc_train[cat_col] = df_rcc_train[cat_col]
df_rcc_train.loc[df_rcc_train[cat_col].isin(top_propensity_products), cat_col] = "TOP_PROPENSITY"
df_rcc_train.loc[df_rcc_train[cat_col].isin(low_propensity_products), cat_col] = "LOW_PROPENSITY"
df_rcc_train.loc[~df_rcc_train[cat_col].isin(list_product), cat_col] = "OTROS"

In [33]:
df_rcc_crosstab = (pd.crosstab([df_rcc_train.key_value, df_rcc_train.codmes], 
                               df_rcc_train[cat_col], 
                               values=df_rcc_train.saldo, 
                               aggfunc="mean")
                    .reset_index(drop=False)
                    .reset_index(drop=True))
df_rcc_crosstab.columns = ["{0}_{1}".format(cat_col, c) if c not in ["key_value", "codmes"] else c for c in df_rcc_crosstab.columns]
feat_cols = [c for c in df_rcc_crosstab.columns if "{0}_".format(cat_col) in c]
df_rcc_crosstab["{0}_ALL_MEAN".format(cat_col)] = df_rcc_crosstab[feat_cols].mean(axis=1)
df_rcc_crosstab["{0}_ALL_SUM".format(cat_col)] = df_rcc_crosstab[feat_cols].sum(axis=1)
df_rcc_crosstab["{0}_ALL_COUNT".format(cat_col)] = df_rcc_crosstab[feat_cols].count(axis=1)

In [34]:
#df_rcc_crosstab.loc[df_rcc_crosstab["PRODUCT_ALL"] > 0].sort_values(by=["PRODUCT_ALL", "codmes"], ascending=[False, False]).head(40)
df_rcc_crosstab.head(10)

,key_value,codmes,PRODUCTO_0,PRODUCTO_1,PRODUCTO_2,PRODUCTO_3,PRODUCTO_4,PRODUCTO_6,PRODUCTO_8,PRODUCTO_OTROS,PRODUCTO_ALL_MEAN,PRODUCTO_ALL_SUM,PRODUCTO_ALL_COUNT
0,0,201703,NaN,-0.072814,-0.038181,-0.041831,NaN,NaN,-0.069253,NaN,-0.055520,-0.222079,4
1,0,201704,NaN,-0.072556,-0.038181,-0.044653,NaN,NaN,-0.066431,-0.072814,-0.058927,-0.294635,5
2,0,201705,NaN,-0.072713,-0.038181,-0.043309,NaN,NaN,-0.067775,-0.072814,-0.058958,-0.294792,5
3,0,201706,NaN,-0.072691,-0.038181,-0.044518,NaN,NaN,-0.066566,-0.072814,-0.058954,-0.294769,5
4,0,201707,NaN,-0.072388,-0.038181,-0.049154,NaN,NaN,-0.061930,-0.072814,-0.058893,-0.294467,5
5,0,201708,NaN,-0.072265,-0.038181,-0.047889,NaN,NaN,-0.063195,-0.072814,-0.058869,-0.294344,5
6,0,201709,0.072673,-0.072052,-0.038181,-0.038181,NaN,NaN,NaN,NaN,-0.018935,-0.075741,4
7,0,201710,0.070836,NaN,-0.038181,-0.038181,NaN,NaN,NaN,NaN,-0.001842,-0.005525,3
8,0,201711,0.070836,-0.071649,-0.038181,-0.038181,NaN,NaN,NaN,NaN,-0.019294,-0.077174,4
9,0,201712,0.064219,NaN,-0.038181,-0.038181,NaN,NaN,NaN,NaN,-0.004048,-0.012143,3


In [15]:
df_rcc_agg = df_rcc_crosstab.groupby(["key_value"], as_index=False).agg({"codmes": ["last", "first"]})
df_rcc_agg.columns = ["_".join(c) if c[-1] != "" else c[0] for c in df_rcc_agg.columns]

In [21]:
def diff_months(codmes_ini, codmes_fin):
  r = relativedelta.relativedelta(datetime.strptime(str(int(codmes_fin)), '%Y%m'), datetime.strptime(str(int(codmes_ini)), '%Y%m'))
  return r.years*12 + r.months

In [22]:
df_rcc_agg["RECENCY"] = df_rcc_agg.apply(lambda row: diff_months(row["codmes_last"], 201803), axis=1)
df_rcc_agg["DIFF_MONTHS_FIRST_LAST"] = df_rcc_agg.apply(lambda row: diff_months(row["codmes_first"], row["codmes_last"]), axis=1)

In [23]:
df_rcc_agg.head()

,key_value,codmes_last,codmes_first,RECENCY,DIFF_MONTHS_FIRST_LAST
0,0,201802,201703,1,11
1,1,201802,201712,1,2
2,2,201802,201703,1,11
3,3,201802,201709,1,5
4,4,201802,201707,1,7


In [24]:
df_rcc_crosstab.columns

Index(['key_value', 'codmes', 'PRODUCT_0', 'PRODUCT_1', 'PRODUCT_2',
       'PRODUCT_3', 'PRODUCT_4', 'PRODUCT_6', 'PRODUCT_8',
       'PRODUCT_LOW_PROPENSITY', 'PRODUCT_OTROS', 'PRODUCT_TOP_PROPENSITY',
       'PRODUCT_ALL'],
      dtype='object')

In [ ]:
"U12M_COUNT_PRODUCT_LOW_PROPENSITY"
"U12M_COUNT_PRODUCT_LOW_PROPENSITY"

In [27]:
round(df_rcc_train["saldo"].min(), 9)

-0.072892241

In [35]:
#df_rcc_train["tipo_dataset"] = "train"
#df_rcc_test["tipo_dataset"] = "test"
del df_rcc_train["tipo_dataset"]
del df_rcc_test["tipo_dataset"]

In [31]:
df_rcc_all = pd.concat([df_rcc_train, df_rcc_test]).reset_index(drop=True)

In [32]:
df_rcc_all["saldo"].mean(), df_rcc_all["saldo"].std()

(0.0005660878377966583, 1.0794203281402588)

In [33]:
df_rcc_all.groupby("tipo_dataset", as_index=False).agg({"saldo": ["mean", "std"]})

tipo_dataset         saldo          
                        mean       std
0         test  1.114166e-03  1.155592
1        train  7.846288e-11  1.000000

In [34]:
del df_rcc_all